In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df['prob_margin'] = np.abs(df['prob1'] - df['prob2'])

In [6]:
df['prob_margin'] = np.abs(df['prob1'] - df['prob2'])
mpm = np.mean(df['prob_margin'])
sdpm = np.std(df['prob_margin'])

In [7]:
prob_margins = []

In [9]:
for x in df['prob_margin']:
    if x < mpm:
        prob_margins.append("MINIMAL PROBABILITY MARGIN")
    elif (x >= mpm) & (x < (mpm + sdpm)):
        prob_margins.append("SLIGHT PROBABILITY ADVANTAGE")
    elif (x >= (mpm + sdpm)) & (x < (mpm + (sdpm * 1.5))):
        prob_margins.append("MODERATE PROBABILITY ADVANTAGE")
    else:
        prob_margins.append("SIGNIFICANT PROBABILITY ADVANTAGE")

In [11]:
df['prob_bucket'] = prob_margins

In [12]:
X_final = df[['prob1','prob2','probtie']]
y_final = df[['prob_bucket']].copy()
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=6)

In [13]:
model = XGBClassifier(n_estimators=400, objective="multiclass:softprob").fit(X_train, y_train.values.ravel())

c:\users\henry\miniconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:56:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [15]:
predictions = model.predict(X_test)

In [16]:
accuracy = accuracy_score(y_test, predictions)

In [17]:
print("Test Accuracy: {}%".format(accuracy * 100))

Test Accuracy: 99.47292764734068%
